In [16]:
# # Import packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from Dataset_HighCorrelationRemoved_ImbalancedCIRemoved import *
# from EDA_script2 import *
# # Set options
# pd.options.display.max_rows = 999
# pd.options.display.max_columns = 999

# train_x_raw = pd.read_csv("../01-Data/X_train.csv", low_memory = True, index_col=0)
# train_y_raw = pd.read_csv("../01-Data/y_train.csv", low_memory = True, index_col=0)
# test_x_raw = pd.read_csv("../01-Data/X_test.csv", low_memory=True, index_col=0)

# df_train = pd.DataFrame(train_x_raw)
# df_test = pd.DataFrame(test_x_raw)
# df_y = pd.DataFrame(train_y_raw)

Add all of the preprocessing below

## Variable 1 - 146 Preprocessing

In [17]:
# columns_to_drop = ['c_abrv', 'f46_IT', 'v72_DE', 'v73_DE', 'v74_DE', 'v75_DE', 'v76_DE', 'v77_DE', 'v78_DE', 'v79_DE']
# df_train.drop(columns=columns_to_drop, inplace=True)
# df_test.drop(columns=columns_to_drop, inplace=True)

## Variable 147 - 292 Preprocessing 

In [18]:
# ### Function to find the targeted colname
# def find_colname(data, target):
#     temp = []
#     for varname in data.columns:
#         if varname.endswith(target):
#             temp.append(varname)
#     return(temp)

# merge_colname = find_colname(train_x_raw, '_11c')
# print(find_colname(train_x_raw, 'c'))
# print(find_colname(train_x_raw, '_r'))

# def merge_columns(dat, colname):
#     for name in colname:
#         name_org = name.replace("_11c", "")
#         dat.loc[dat[name_org] == -4, name_org] = dat.loc[dat[name_org] == -4, name]

## Variable 293 - 438 Preprocessing

In [19]:
# ## removed string type data
# df_train.drop('v228b', inplace=True, axis=1) 
# df_test.drop('v228b', inplace=True, axis=1) 

# df_train.fillna({'v228b_r': -3}, inplace=True)
# df_test.fillna({'v228b_r': -3}, inplace=True)

# df_train.drop('v231b', inplace=True, axis=1) 
# df_test.drop('v231b', inplace=True, axis=1)

# df_train.fillna({'v231b_r': -3}, inplace=True)
# df_test.fillna({'v231b_r': -3}, inplace=True)

# df_train.drop('v233b', inplace=True, axis=1)
# df_test.drop('v233b', inplace=True, axis=1)

# df_train.fillna({'v233b_r': -3}, inplace=True)
# df_test.fillna({'v233b_r': -3}, inplace=True)

# df_train.drop('v251b', inplace=True, axis=1)
# df_test.drop('v251b', inplace=True, axis=1) 

# df_train.fillna({'v251b_r': -3}, inplace=True)
# df_test.fillna({'v251b_r': -3}, inplace=True)

# df_train.drop('f252_edulvlb_CH', inplace=True, axis=1)
# df_test.drop('f252_edulvlb_CH', inplace=True, axis=1)

# ## removed the column having 'DE'
# df_train.drop(list(df_train.filter(regex='DE')), axis=1, inplace=True)
# df_test.drop(list(df_test.filter(regex='DE')), axis=1, inplace=True)

# ## removed the column having 'GB'
# df_train.drop(list(df_train.filter(regex='GB')), axis=1, inplace=True)
# df_test.drop(list(df_test.filter(regex='GB')), axis=1, inplace=True)

# df_train.drop('v281a', inplace=True, axis=1)
# df_test.drop('v281a', inplace=True, axis=1)

# df_train.drop('v275b_N2', inplace=True, axis=1) 
# df_test.drop('v275b_N2', inplace=True, axis=1) 

# df_train.drop('v275b_N1', inplace=True, axis=1) 
# df_test.drop('v275b_N1', inplace=True, axis=1) 

# Simple Model Run
##  xgBoost model set up

In [20]:
# one-hot encoding

# Apply get_dummies to the entire DataFrame, automatically encoding all categorical columns
df_train_encoded = pd.get_dummies(df_train)
df_test_encoded = pd.get_dummies(df_test)

# To ensure the training and test sets have the same columns after encoding, you might need to align them
df_train_encoded, df_test_encoded = df_train_encoded.align(df_test_encoded, join='left', axis=1, fill_value=0)

In [21]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import log_loss

label_mapping = {-1: 0, 1: 1, 2: 2, 3: 3, 4: 4}
df_y = df_y.replace(label_mapping)

dtrain = xgb.DMatrix(df_train, label=df_y, enable_categorical=True)
dtest = xgb.DMatrix(df_test, enable_categorical=True)

params = {
    'max_depth': 6,
    'colsample_bytree': 0.8,
    'gamma': 0.3,
    #'n_estimators': 1000,
    'subsample': 0.7,
    'eta': 0.01,
    'objective': 'multi:softprob',
    'num_class': 5,
    'eval_metric': 'mlogloss',
    'lambda': 1,  # Added L2 regularization
    'alpha': 0.1  # Added L1 regularization
}

num_boost_round = 1000

## Cross Validation

In [22]:
from xgboost import cv

# params and num_boost_round provided above
# xgb_cv = cv(dtrain=dtrain, params=params, nfold=5,
#             num_boost_round=num_boost_round, early_stopping_rounds=10,
#             metrics="mlogloss", as_pandas=True, seed=123)
# 
# xgb_cv

In [23]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Assuming df_train is your features and df_y is your labels
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df_train, df_y, test_size=0.3, random_state=42)

# Create DMatrix for training and validation
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dval = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

# Container for evaluation results
evals_result = {}

# Train the model
bst = xgb.train(params, dtrain, num_boost_round,
                    evals=[(dtrain, 'train'), (dval, 'val')],
                    evals_result=evals_result,
                    early_stopping_rounds=10,
                    verbose_eval=True)

# Retrieve the last evaluation metric for both train and val sets
train_last_eval = evals_result['train']['mlogloss'][-1]
val_last_eval = evals_result['val']['mlogloss'][-1]

print(f"Training Multiclass Logarithmic Loss: {train_last_eval}")
print(f"Validation Multiclass Logarithmic Loss: {val_last_eval}")


[0]	train-mlogloss:1.60114	val-mlogloss:1.60156
[1]	train-mlogloss:1.59319	val-mlogloss:1.59404
[2]	train-mlogloss:1.58532	val-mlogloss:1.58654
[3]	train-mlogloss:1.57759	val-mlogloss:1.57916
[4]	train-mlogloss:1.57004	val-mlogloss:1.57200
[5]	train-mlogloss:1.56226	val-mlogloss:1.56462
[6]	train-mlogloss:1.55484	val-mlogloss:1.55764
[7]	train-mlogloss:1.54741	val-mlogloss:1.55058
[8]	train-mlogloss:1.54001	val-mlogloss:1.54361
[9]	train-mlogloss:1.53330	val-mlogloss:1.53725
[10]	train-mlogloss:1.52650	val-mlogloss:1.53078
[11]	train-mlogloss:1.51960	val-mlogloss:1.52429
[12]	train-mlogloss:1.51291	val-mlogloss:1.51794
[13]	train-mlogloss:1.50609	val-mlogloss:1.51152
[14]	train-mlogloss:1.49939	val-mlogloss:1.50515
[15]	train-mlogloss:1.49274	val-mlogloss:1.49885
[16]	train-mlogloss:1.48627	val-mlogloss:1.49279
[17]	train-mlogloss:1.47991	val-mlogloss:1.48683
[18]	train-mlogloss:1.47366	val-mlogloss:1.48092
[19]	train-mlogloss:1.46733	val-mlogloss:1.47493
[20]	train-mlogloss:1.46124	va

## xgboost train

In [24]:
evals_result = {}
bst = xgb.train(params, dtrain, num_boost_round, 
                evals=[(dtrain, 'train')], evals_result=evals_result, 
                verbose_eval=False)
print(f"Training Multiclass Logarithmic Loss: {evals_result['train']['mlogloss'][-1]}")

y_test_probs = bst.predict(dtest)

class_order = [0, 1, 2, 3, 4]
class_mapping = {class_label: f"Class_{class_label}" for class_label in class_order}
df_train_cvt = xgb.DMatrix(df_train, label=df_y, enable_categorical=True)
y_train_probs = bst.predict(df_train_cvt)

val_log_loss = log_loss(df_y['label'], y_train_probs, labels=class_order)
print(f"Validation Multiclass Logarithmic Loss: {val_log_loss}")

Training Multiclass Logarithmic Loss: 0.6165175621576297
Validation Multiclass Logarithmic Loss: 0.6902145717528759


## Learning Curves

Training Multiclass Logarithmic Loss: 0.5112099582428733
Validation Multiclass Logarithmic Loss: 0.5112099476073049


## Generate Submission csv

In [ ]:
submission_df = pd.DataFrame(y_test_probs, columns=class_mapping.values())
submission_df.columns = ['no answer', 'very important', 'quite important', 'not important', 'not at all important']
submission_df.insert(0, 'id', df_test.index)

# Save the submission file
submission_file = ('test_submission.csv')
submission_df.to_csv(submission_file, index=False)

In [ ]:
df_train_encoded

,year,fw_start_month,fw_duration,country,v1,v2,v3,v4,v5,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,f20,v20,v20a,v20b,v21,v22,v23,v24,f24_IT,v24a_IT,v24b_IT,v25,v26,v27,v28,v29,v30,f30a,v30a,v30b,v30c,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,f45a,v45a,v45b,v45c,v46,v47,v48,v49,v50,v51,v52,v52_cs,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,f85,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,f96,v96,v96a,v96b,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,f108,v108,v109,f110,v110,v111,v111_4,v112,f112_SE,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v133_11c,v134,v134_11c,v135,v135_11c,v136,v136_11c,v137,v137_11c,v138,v138_11c,v139,v139_11c,v140,v140_11c,v141,v141_11c,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174_LR,v174_cs,v175_LR,v175_cs,v176,v176_DK,v177,v177_DK,v178,v178_DK,v179,v179_DK,v180,v180_DK,v181,v181_DK,v182,v182_DK,v183,v183_DK,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v221_DK,v222,v222_DK,v223,v223_DK,v224,v224_DK,v225,v226,age,age_r,age_r2,age_r3,v227,v228b_r,v229,v230,v231b_r,v232,v233b_r,v234,v235,v236,v237,v238,v239_r,v239a,v239b,v240,v241,v242,v242_r,v243_ISCED_3,v244,v245,v246_ESeC,v247,v248,v248a,v249,v250,v251b_r,v252_edulvlb_2,v253,v254,v255_ESeC,v256,v257,v258,v259,v260,v261,v261_r,v262_edulvlb_2,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275c_N1,v276_r,v278a,v279d_r,v280,v281a_r,v282
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2020,11,0,804,2,1,2,2,3,2,3,1,2,1,2,2,2,2,2,2,2,2,0,2,2,2,1,2,1,2,-4,-4,-4,1,1,2,2,2,1,-4,-4,-4,-4,2,1,2,2,3,2,2,5,5,1,1,2,1,1,1,0,2,-4,-4,1,2,2,3,3,1,1,-100,-3,2,2,1,1,1,1,1,1,1,9,1,1,2,2,2,1,2,2,2,2,2,2,2,3,2,2,1,2,4,3,2,0,2,1,1,2,2,1,2,1,1,2,2,0,2,-4,-4,3,2,2,2,3,-2,3,8,3,5,5,0,2,1,0,4,2,3,1,-4,3,3,1,1,2,2,3,3,4,3,3,3,3,3,4,3,3,3,3,3,7,-4,0,-4,10,-4,9,-4,0,-4,10,-4,5,-4,0,-4,10,-4,10,5,1,4,3,4,1,2,2,1,2,1,4,4,5,1,5,2,1,5,1,3,1,1,1,1,1,1,2,1,1,-4,5,-100,8,-100,1,-4,3,-4,2,-4,3,-4,2,-4,1,-4,3,-4,4,-4,3,5,3,3,5,3,1,1,1,1,3,2,3,1,3,2,4,4,4,4,1,2,4,4,2,1,4,5,3,3,3,4,4,2,2,5,2,2,-4,2,-4,1,-4,1,-4,1,1960,60,5,3,5,1,-3.0,-3,1,-3.0,1,-3.0,1,2,-3,-3,1,0,0,0,2,58,19,8,354,5,1,9,-3,2,-3,2,1,-3.0,42,9,1,1,-3,1,1,2,2,2,1,31,1,1,9,2,4,3,4,3,4,2,4,80401,1,14,55,1,475,32
1,2018,9,2,688,2,1,2,2,3,2,2,2,2,2,2,2,2,1,2,2,2,2,0,2,-4,-4,2,2,1,2,-4,-4,-4,1,1,2,2,2,2,-4,-4,-4,-4,2,1,2,2,3,-1,-1,9,8,1,1,2,2,2,1,3,-2,-4,-4,1,2,1,2,3,2,-3,-3,2,-2,-2,-2,-2,-2,-2,-2,-2,3,-2,-2,1,2,2,1,1,2,2,2,2,2,3,4,4,4,2,3,4,5,2,1,0,1,2,1,2,2,1,1,2,2,2,1,0,2,-4,-4,3,3,3,3,3,-2,2,1,2,1,9,0,2,4,0,3,1,1,-2,-4,2,1,2,2,3,4,4,3,4,3,4,4,4,3,2,3,3,4,4,4,7,-4,4,-4,5,-4,6,-4,4,-4,5,-4,4,-4,7,-4,5,-4,5,5,3,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,1,3,3,1,2,3,2,-4,-2,-2,-2,-2,-2,-4,-2,-4,-2,-4,-2,-4,-2,-4,-2,-4,-2,-4,-2,-4,3,4,5,4,7,2,2,2,2,2,2,2,2,2,7,2,4,5,4,4,1,-1,-1,-1,3,4,2,2,2,3,3,4,3,2,2,3,2,2,-4,2,-4,1,-4,1,-4,2,1976,42,3,2,3,1,-3.0,-3,1,-3.0,1,-3.0,1,1,1,-3,2,2,2,0,6,14,19,8,354,1,-3,1,-3,2,-3,2,1,-3.0,32,8,1,-3,-3,2,-3,2,2,7,3,31,1,1,3,1,3,2,4,2,3,3,2,68801,3,17,73,2,397,3
2,2017,6,9,352,2,2,1,1,2,2,1,2,2,1,2,2,2,1,1,2,2,2,0,2,-4,-4,1,2,1,2,-4,-4,-4,1,2,2,2,2,2,-4,-4,-4,-4,1,1,2,1,2,2,2,7,9,1,1,1,1,1,1,0,2,-4,-4,5,4,5,5,5,2,-3,-3,1,7,6,3,2,-1,2,2,-1,3,3,7,1,2,2,1,2,1,1,4,4,4,4,4,4,4,2,5,5,1,5,2,0,2,1,2,1,1,1,2,1,2,2,2,0,2,-4,-4,1,1,1,1,2,4,8,8,2,4,7,0,3,4,0,2,4,3,-1,-4,1,2,3,-4,2,3,2,3,3,2,

In [ ]:
df_train

,year,fw_start_month,fw_duration,country,v1,v2,v3,v4,v5,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,f20,v20,v20a,v20b,v21,v22,v23,v24,f24_IT,v24a_IT,v24b_IT,v25,v26,v27,v28,v29,v30,f30a,v30a,v30b,v30c,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,f45a,v45a,v45b,v45c,v46,v47,v48,v49,v50,v51,v52,v52_cs,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,f85,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,f96,v96,v96a,v96b,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,f108,v108,v109,f110,v110,v111,v111_4,v112,f112_SE,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v133_11c,v134,v134_11c,v135,v135_11c,v136,v136_11c,v137,v137_11c,v138,v138_11c,v139,v139_11c,v140,v140_11c,v141,v141_11c,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174_LR,v174_cs,v175_LR,v175_cs,v176,v176_DK,v177,v177_DK,v178,v178_DK,v179,v179_DK,v180,v180_DK,v181,v181_DK,v182,v182_DK,v183,v183_DK,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v221_DK,v222,v222_DK,v223,v223_DK,v224,v224_DK,v225,v226,age,age_r,age_r2,age_r3,v227,v228b_r,v229,v230,v231b_r,v232,v233b_r,v234,v235,v236,v237,v238,v239_r,v239a,v239b,v240,v241,v242,v242_r,v243_ISCED_3,v244,v245,v246_ESeC,v247,v248,v248a,v249,v250,v251b_r,v252_edulvlb_2,v253,v254,v255_ESeC,v256,v257,v258,v259,v260,v261,v261_r,v262_edulvlb_2,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275c_N1,v276_r,v278a,v279d_r,v280,v281a_r,v282
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2020,11,0,804,2,1,2,2,3,2,3,1,2,1,2,2,2,2,2,2,2,2,0,2,2,2,1,2,1,2,-4,-4,-4,1,1,2,2,2,1,-4,-4,-4,-4,2,1,2,2,3,2,2,5,5,1,1,2,1,1,1,0,2,-4,-4,1,2,2,3,3,1,1,-100,-3,2,2,1,1,1,1,1,1,1,9,1,1,2,2,2,1,2,2,2,2,2,2,2,3,2,2,1,2,4,3,2,0,2,1,1,2,2,1,2,1,1,2,2,0,2,-4,-4,3,2,2,2,3,-2,3,8,3,5,5,0,2,1,0,4,2,3,1,-4,3,3,1,1,2,2,3,3,4,3,3,3,3,3,4,3,3,3,3,3,7,-4,0,-4,10,-4,9,-4,0,-4,10,-4,5,-4,0,-4,10,-4,10,5,1,4,3,4,1,2,2,1,2,1,4,4,5,1,5,2,1,5,1,3,1,1,1,1,1,1,2,1,1,-4,5,-100,8,-100,1,-4,3,-4,2,-4,3,-4,2,-4,1,-4,3,-4,4,-4,3,5,3,3,5,3,1,1,1,1,3,2,3,1,3,2,4,4,4,4,1,2,4,4,2,1,4,5,3,3,3,4,4,2,2,5,2,2,-4,2,-4,1,-4,1,-4,1,1960,60,5,3,5,1,-3.0,-3,1,-3.0,1,-3.0,1,2,-3,-3,1,0,0,0,2,58,19,8,354,5,1,9,-3,2,-3,2,1,-3.0,42,9,1,1,-3,1,1,2,2,2,1,31,1,1,9,2,4,3,4,3,4,2,4,80401,1,14,55,1,475,32
1,2018,9,2,688,2,1,2,2,3,2,2,2,2,2,2,2,2,1,2,2,2,2,0,2,-4,-4,2,2,1,2,-4,-4,-4,1,1,2,2,2,2,-4,-4,-4,-4,2,1,2,2,3,-1,-1,9,8,1,1,2,2,2,1,3,-2,-4,-4,1,2,1,2,3,2,-3,-3,2,-2,-2,-2,-2,-2,-2,-2,-2,3,-2,-2,1,2,2,1,1,2,2,2,2,2,3,4,4,4,2,3,4,5,2,1,0,1,2,1,2,2,1,1,2,2,2,1,0,2,-4,-4,3,3,3,3,3,-2,2,1,2,1,9,0,2,4,0,3,1,1,-2,-4,2,1,2,2,3,4,4,3,4,3,4,4,4,3,2,3,3,4,4,4,7,-4,4,-4,5,-4,6,-4,4,-4,5,-4,4,-4,7,-4,5,-4,5,5,3,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,1,3,3,1,2,3,2,-4,-2,-2,-2,-2,-2,-4,-2,-4,-2,-4,-2,-4,-2,-4,-2,-4,-2,-4,-2,-4,3,4,5,4,7,2,2,2,2,2,2,2,2,2,7,2,4,5,4,4,1,-1,-1,-1,3,4,2,2,2,3,3,4,3,2,2,3,2,2,-4,2,-4,1,-4,1,-4,2,1976,42,3,2,3,1,-3.0,-3,1,-3.0,1,-3.0,1,1,1,-3,2,2,2,0,6,14,19,8,354,1,-3,1,-3,2,-3,2,1,-3.0,32,8,1,-3,-3,2,-3,2,2,7,3,31,1,1,3,1,3,2,4,2,3,3,2,68801,3,17,73,2,397,3
2,2017,6,9,352,2,2,1,1,2,2,1,2,2,1,2,2,2,1,1,2,2,2,0,2,-4,-4,1,2,1,2,-4,-4,-4,1,2,2,2,2,2,-4,-4,-4,-4,1,1,2,1,2,2,2,7,9,1,1,1,1,1,1,0,2,-4,-4,5,4,5,5,5,2,-3,-3,1,7,6,3,2,-1,2,2,-1,3,3,7,1,2,2,1,2,1,1,4,4,4,4,4,4,4,2,5,5,1,5,2,0,2,1,2,1,1,1,2,1,2,2,2,0,2,-4,-4,1,1,1,1,2,4,8,8,2,4,7,0,3,4,0,2,4,3,-1,-4,1,2,3,-4,2,3,2,3,3,2,